In [13]:
import sys
from pathlib import Path

_BASE_DIR = Path().resolve().parent.parent
print("(!) Make sure this dir is project directory: ", _BASE_DIR)
sys.path.append(str(_BASE_DIR))

from utils.settings import get_in_out_dirs

# -----------------------------
# 1. Папки данных
# -----------------------------

_LAB_NAME = "lab9"

INPUT_DIR, OUTPUT_DIR = get_in_out_dirs(base_name=_LAB_NAME)

DATA_DIR = INPUT_DIR / "data_2"
DATA_SRC_DIR = INPUT_DIR / "data_2_src"
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"

DATA_DIR.mkdir(parents=True, exist_ok=True)
DATA_SRC_DIR.mkdir(parents=True, exist_ok=True)
TRAIN_DIR.mkdir(parents=True, exist_ok=True)
TEST_DIR.mkdir(parents=True, exist_ok=True)

(!) Make sure this dir is project directory:  /home/ars/Desktop/university/bsu-nn


# Лаб. 9

## Задание 2.  (из  темы  9.2):

Дообучите нейронную сеть ResNet34 распознаванию Вашего лица на фотографии. Используйте дообученную сеть для распознавания Вашего лица на других фотографиях (другой возраст; другой ракурс; часть лица закрыта, например, очками).

In [8]:
import kagglehub
import shutil

# Download latest version
path = kagglehub.dataset_download("vasukipatel/face-recognition-dataset")
print("Dataset path:", path)

# Копируем в DATA_SRC_DIR
for item in Path(path).iterdir():
    dest = DATA_SRC_DIR / item.name
    if item.is_dir():
        shutil.copytree(item, dest, dirs_exist_ok=True)
    else:
        shutil.copy(item, dest)

Dataset path: /home/ars/.cache/kagglehub/datasets/vasukipatel/face-recognition-dataset/versions/1


In [15]:
# -----------------------------
# 2. Подготовка датасета по CSV
# -----------------------------
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

# Читаем CSV
csv_path = DATA_DIR_SRC / "Dataset.csv"
df = pd.read_csv(csv_path)

# Создаём train/test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Копируем файлы в структуру папок для ImageDataGenerator
for subset_df, subset_dir in [(train_df, TRAIN_DIR), (test_df, TEST_DIR)]:
    for _, row in subset_df.iterrows():
        label_dir = subset_dir / row['label']
        label_dir.mkdir(parents=True, exist_ok=True)
        # файл в Faces/Faces/
        src_file = DATA_DIR_SRC / "Faces" / "Faces" / row['id']
        dst_file = label_dir / row['id']
        shutil.copy(src_file, dst_file)


In [17]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 749.9 kB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.3 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 4.0 MB/s eta 0:00:004.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 3.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 4.9 MB/s eta 0:00:00m eta 0:00:010:00:01


In [26]:
# -----------------------------
# 3. Архитектура модели (PyTorch)
# -----------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

target_size_tp = 128
batch_size = 16

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((target_size_tp, target_size_tp)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((target_size_tp, target_size_tp)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Datasets
train_dataset = ImageFolder(TRAIN_DIR, transform=train_transform)
val_dataset = ImageFolder(TRAIN_DIR, transform=val_transform)
test_dataset = ImageFolder(TEST_DIR, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model
model = timm.create_model('resnet34', pretrained=True, num_classes=len(train_dataset.classes))
model = model.to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [27]:
train_dataset.classes

['Akshay Kumar',
 'Alexandra Daddario',
 'Alia Bhatt',
 'Amitabh Bachchan',
 'Andy Samberg',
 'Anushka Sharma',
 'Billie Eilish',
 'Brad Pitt',
 'Camila Cabello',
 'Charlize Theron',
 'Claire Holt',
 'Courtney Cox',
 'Dwayne Johnson',
 'Elizabeth Olsen',
 'Ellen Degeneres',
 'Henry Cavill',
 'Hrithik Roshan',
 'Hugh Jackman',
 'Jessica Alba',
 'Kashyap',
 'Lisa Kudrow',
 'Margot Robbie',
 'Marmik',
 'Natalie Portman',
 'Priyanka Chopra',
 'Robert Downey Jr',
 'Roger Federer',
 'Tom Cruise',
 'Vijay Deverakonda',
 'Virat Kohli',
 'Zac Efron']

In [28]:
MODEL_DIR = OUTPUT_DIR / "model"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

OUT_MODEL_PATH = MODEL_DIR / "resnet34_faces.pth"

In [29]:
# -----------------------------
# 4. Обучение или загрузка
# -----------------------------

if OUT_MODEL_PATH.exists():
    print("Модель найдена, загружаем вместо тренировки...")
    model.load_state_dict(torch.load(OUT_MODEL_PATH, map_location=device))
    model.to(device)

else:
    print("Модель не найдена, тренируем...")
    num_epochs = 1  # для теста
    log_interval = 10  # печатать каждые 10 батчей

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if (batch_idx + 1) % log_interval == 0:
                print(f"Epoch {epoch+1} [{batch_idx+1}/{len(train_loader)}], "
                      f"Loss: {running_loss/(batch_idx+1):.4f}, "
                      f"Accuracy: {correct/total:.4f}")

        # Лог по эпохе
        print(f"Epoch {epoch+1} finished, Loss: {running_loss/len(train_loader):.4f}, "
              f"Accuracy: {correct/total:.4f}")

    print("Сохраняем модель...")
    torch.save(model.state_dict(), OUT_MODEL_PATH)


Модель не найдена, тренируем...


Epoch 1 [10/129], Loss: 3.4506, Accuracy: 0.0187
Epoch 1 [20/129], Loss: 3.4258, Accuracy: 0.0312
Epoch 1 [30/129], Loss: 3.4213, Accuracy: 0.0396
Epoch 1 [40/129], Loss: 3.4124, Accuracy: 0.0391
Epoch 1 [50/129], Loss: 3.4099, Accuracy: 0.0387
Epoch 1 [60/129], Loss: 3.4051, Accuracy: 0.0365
Epoch 1 [70/129], Loss: 3.3989, Accuracy: 0.0446
Epoch 1 [80/129], Loss: 3.3917, Accuracy: 0.0523
Epoch 1 [90/129], Loss: 3.3837, Accuracy: 0.0590
Epoch 1 [100/129], Loss: 3.3792, Accuracy: 0.0625
Epoch 1 [110/129], Loss: 3.3732, Accuracy: 0.0659
Epoch 1 [120/129], Loss: 3.3701, Accuracy: 0.0677
Epoch 1 finished, Loss: 3.3651, Accuracy: 0.0708
Сохраняем модель...


In [30]:
# -----------------------------
# 5. Оценка модели на тесте
# -----------------------------

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {correct/total:.4f}")

Test Accuracy: 0.1404
